# **The HandTracker class**

This notebook provides some documentation about ```HandTracker``` objects.

Run the following codeblocks to import the ```HandTracker``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Programming\Game-Development\fivecarddraw


In [2]:
from fivecarddraw import HandTracker

## **Initialisation**

The ```HandTracker``` class is designed to deal with card dynamics, and is one of the trackers mediated by the ```Dealer``` class. Upon initialisation:
* It creates a ```Deck``` in the attribute ```HandTracker.DECK```.
* It creates a ```dict``` that stores data about players hands as json in the attribute ```HandTracker.players```
* It loads about 150kb of data from the [data](../data/) folder that contains data useful for evaluating hands. This data is loaded using the ```HandTracker.LoadData()``` method and stored in ```dict``` attributes, namely: ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS```, and ```HandTracker.DUPE_RANKS```.

For more information about ```Deck``` objects, see the [deck.ipynb](deck.ipynb) notebook. For more information about ```Dealer``` objects see the [dealer.ipynb](dealer.ipynb) notebook.


In [3]:
tracker = HandTracker()
print(f"deck: {tracker.DECK}\n")
print(f"hands: {tracker.players}\n")
print(f"Amount of keys in FLUSH_RANKS: {len(tracker.FLUSH_RANKS)}")
print(f"Amount of keys in UNIQUE5_RANKS: {len(tracker.UNIQUE5_RANKS)}")
print(f"Amount of keys in DUPE_RANKS: {len(tracker.DUPE_RANKS)}\n")

deck: [2♠, 2♡, 2♢, 2♣, 3♠, 3♡, 3♢, 3♣, 4♠, 4♡, 4♢, 4♣, 5♠, 5♡, 5♢, 5♣, 6♠, 6♡, 6♢, 6♣, 7♠, 7♡, 7♢, 7♣, 8♠, 8♡, 8♢, 8♣, 9♠, 9♡, 9♢, 9♣, 10♠, 10♡, 10♢, 10♣, J♠, J♡, J♢, J♣, Q♠, Q♡, Q♢, Q♣, K♠, K♡, K♢, K♣, A♠, A♡, A♢, A♣]

hands: {}

Amount of keys in FLUSH_RANKS: 1287
Amount of keys in UNIQUE5_RANKS: 1287
Amount of keys in DUPE_RANKS: 4888



### **Loading Hand Evaluation Data**

The folder [data](../data/) contains three different files which are tables of data that contain both numerical and categorical evaluations of every possible five card draw hand. The actual data in those files was scraped from an external website [here](http://suffe.cool/poker/7462.html). The purpose of the ```HandTracker.LoadData()``` method is to load this data into the ```HandTracker``` object so that it can be used to quickly evaluate hands. 

Each file in the [data](../data/) folder is formatted in the same way. The lines can be interpreted as follows:

* The first two letters encode a categorical evaluation; either RF, SF, 4K, FH, FF, SS, 3K, 2P, 1P or HC which stand for royal flush, straight flush, four of a kind, full house, flush, straight, three of a kind, two pair, pair or high card respectively.
* The next five characters give the cards in a hand; either A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2.
* The remaining numbers give a numerical evaluation; 1 being the best and 7462 being the worst. These ordinals were calculated through combinatorial analysis by [Kevin Suffecool](http://suffe.cool/poker/evaluator.html), the owner of the site where the data was scraped.

In [10]:
tracker = HandTracker()

with open("data/flushes.txt", "r") as flush_data:
    line = flush_data.readline()[:-1]
    print(f"data\\flushes.txt data sample:\n{line}\n")

with open("data/uniquefive.txt", "r") as unique5_data:
    line = unique5_data.readline()[:-1]
    print(f"data\\uniquefive.txt data sample:\n{line}\n")

with open("data/dupes.txt", "r") as dupe_data:
    line = dupe_data.readline()[:-1]
    print(f"data\\dupes.txt data sample:\n{line}")

data\flushes.txt data sample:
RF  AKQJT  1

data\uniquefive.txt data sample:
SS  AKQJT  1600

data\dupes.txt data sample:
4K  AAAAK  11


The ```HandTracker.LoadData()``` method reads each hand and converts it into an ```int``` that represents the hand irrepective of the order of cards in the hand. 

* For hands with no duplicates, the hand is converted to an ```int``` using binary arithmetic, with cards represented as powers of two and added together to encode the hand.
* For hands with duplicates, the hand is converted to an ```int``` with a unique prime factorisation, with cards represented as prime numbers and multiplied together to encode the hand.

The ```int```  is then used as a key for either ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS```, and ```HandTracker.DUPE_RANKS``` depending on the type of hand. The associated value for the key is a ```list``` which contains the numerical rank of the hand from 1-7462, and a categorical evaluation. 

In [12]:
tracker = HandTracker()

hand_encoding = 37 * 41 ** 4
print(f"AAAAK encoded: {hand_encoding}")
print(f"AAAAK rank_n: {tracker.DUPE_RANKS[hand_encoding][0]}")
print(f"AAAAK rank_c: {tracker.DUPE_RANKS[hand_encoding][1]}")

AAAAK encoded: 104553157
AAAAK rank_n: 11
AAAAK rank_c: four of a kind


## **Player Tracking**

The ```HandTracker.TrackPlayers(names)``` method is used to begin tracking the hands of a player. The similarly named ```HandTracker.TrackedPlayers()``` is used to get the names of players being tracked, by returning the ```dict_keys``` of the ```HandTracker.players``` attribute.

In [23]:
tracker = HandTracker()
names = ["Brad"]

tracker.TrackPlayers(names)
print(f"Player info: {tracker.players}\n")
print(f"Tracked players: {tracker.TrackedPlayers()}")

Player info: {'Brad': {'cards': []}}

Tracked players: dict_keys(['Brad'])


There are two methods for modifying a ```list``` of cards stored in ```HandTracker.players```.

* ```HandTracker.AssignCards(name, cards)``` is used to append cards.
* ```HandTracker.UnassignCards(name, cards)``` is used to remove cards.

Both methods will throw an ```Exception``` if a name is used that isn't being tracked. The ```HandTracker.UnassignCards(name, cards)``` will also throw an ```Exception``` if the cards parameter contains a ```Card``` that isn't assigned to player with name. It's not recommended to explicitly use these methods as they ignores deck tracking. For higher-level methods see the sections on dealing and swapping cards below. To view a single players cards, use ```HandTracker.TrackedHand(name)```

In [3]:
from fivecarddraw import Card

In [4]:
tracker = HandTracker()
names = ["Brad"]
tracker.TrackPlayers(names)
print(f"hands before assignment: {tracker.players}\n")

cards = [Card(12-i, 0) for i in range(5)]
tracker.AssignCards(names[0], cards)
print(f"hands after assignment: {tracker.players}\n")

tracker.UnassignCards(names[0], cards[1:])
print(f"hands after unassignment: {tracker.players}")

hands before assignment: {'Brad': {'cards': []}}

hands after assignment: {'Brad': {'cards': [A♠, K♠, Q♠, J♠, 10♠]}}

hands after unassignment: {'Brad': {'cards': [A♠]}}


## **Dealing Hands**

The ```HandTracker.DealHand()``` method can be used to obtain a five card hand, taken from ```HandTracker.DECK```. For this reason, it asserts there ar enough cards in the deck to deal. It's recommended to shuffle cards before dealing! Shuffling can be simulated with the convenience method ```HandTracker.ShuffleDeck()```. 

In [8]:
tracker = HandTracker()
tracker.ShuffleDeck()
hand = tracker.DealHand()
print(f"Hand: {hand}")

Hand: [Q♢, 5♢, 8♡, A♣, 2♢]


If player hand tracking is desired, the recommended way to deal cards is using the ```HandTracker.DealPlayersIn()``` method, which also updates the   ```HandTracker.players``` attribute.

In [4]:
tracker = HandTracker()
names = ["Brad", "Scott", "Dan", "Alex", "Olivia", "Lewis"]
tracker.TrackPlayers(names)

tracker.ShuffleDeck()
tracker.DealPlayersIn()
print(f"player hands: {tracker.players}")

player hands: {'Brad': {'cards': [8♡, 2♡, 7♢, 10♢, 7♡]}, 'Scott': {'cards': [7♣, 5♡, 3♣, 7♠, 3♢]}, 'Dan': {'cards': [2♠, 5♠, A♠, 10♣, 3♠]}, 'Alex': {'cards': [J♠, 9♢, K♠, Q♠, 9♠]}, 'Olivia': {'cards': [6♡, Q♢, 4♠, 5♢, 8♣]}, 'Lewis': {'cards': [10♠, J♣, Q♡, 9♣, K♢]}}


## **Evaluating Hands**

The ```HandTracker.EvaluateHand()``` method can be used to evaluate a hand. It does the following:

* It determines which of the ```dict``` attributes contain the rank of the hand; namely: ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` or ```HandTracker.DUPE_RANKS```.
* It converts the hand composition into an ```int```, using an algorithm dependent on which ```dict``` the hands rank is stored.
* The ```int``` representing the hand is used as a key in the respective ```dict``` attribute, to retrieve both the numerical rank of the hand, and it's categorical rank.
* It returns the hand information.

There are 7462 different ranks in total, as calculated by [Kevin Suffecool](http://suffe.cool/poker/evaluator.html), with 1 being the best hand and 7462 being the worst. The categorical ranks are: 'royal flush', 'straight flush', 'four of a kind', 'full house', 'flush', 'straight', 'three of a kind', 'two pair', 'pair' or 'high card'.

In [5]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()
print(f"Hand: {hand}")
rank = tracker.EvaluateHand(hand)
print(f"numerical rank: {rank[0]}")
print(f"categorical rank: {rank[1]}")

Hand: [2♡, 8♡, K♢, 2♢, 3♠]
numerical rank: 6055
categorical rank: pair


### **Locating Hand Rank**

Before the rank of a hand can pulled from one of the ```dict``` attributes of the ```HandTracker``` class, there must be an inspection of the hand to determine which of the ```dict``` attributes the ranking is stored in.

* If ```HandTracker.HasFlush(hand) == True``` then the hand's ranking is in ```HandTracker.FLUSH_RANKS```.
* Else if ```HandTracker.HasUnique5(hand) == True``` then the hand's ranking is in ```HandTracker.UNIQUE5_RANKS```.
* Else the hand's ranking is in ```HandTracker.DUPES_RANKS```.

In [12]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()

if tracker.HasFlush(hand):
    print(f"{hand} contains a flush.")
elif tracker.HasUnique5(hand):
    print(f"{hand} contains five unique valued cards.")
else:
    print(f"{hand} contains duplicate valued cards.")

[5♢, 8♡, J♡, 10♠, 10♢] contains duplicate valued cards.


The ```HandTracker.HasFlush()``` method involves bitwise-and of the ```int(Card)``` values of the cards in the hand. The ```int(Card)``` value of a card is a binary encoding, shaped by a bitmask that assigns each unique card suit to a power of two. As long as each card in the hand has the same suit, the result of bitwise-and will produce an ```int``` with one on-bit.

In [27]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()

hand_int = 0
for card in hand:
    print(f"{card} suit: {bin(card._suit >> 12)[2:].zfill(4)}")
    hand_int &= card._suit >> 12
print(f"bitwise-and: {bin(hand_int)[2:].zfill(4)}")
print(f"has flush: {tracker.HasFlush(hand)}")

2♡ suit: 0010
J♣ suit: 1000
2♢ suit: 0100
K♠ suit: 0001
5♢ suit: 0100
bitwise-and: 0000
has flush: False


The ```HandTracker.HasUnique5()``` method involves bitwise-xor of the ```int(Card)``` values of the cards in the hand. The ```int(Card)``` value of a card is a binary encoding, shaped by a bitmask that assigns each unique card value to a power of two. As long as each card in the hand has a different value, the result of bitwise-xor will produce an ```int``` with five on-bits.

In [31]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()

hand_int = 0
for card in hand:
    print(f"{card} value: {bin(card._value >> 16)[2:].zfill(13)}")
    hand_int |= card._value >> 16
print(f"bitwise-xor: {bin(hand_int)[2:].zfill(13)}")
print(f"has no dupes: {tracker.HasUnique5(hand)}")

9♠ value: 0000010000000
5♡ value: 0000000001000
J♡ value: 0001000000000
3♢ value: 0000000000010
2♣ value: 0000000000001
bitwise-xor: 0001010001011
has no dupes: True


### **Encoding Hand as integer**

Once the location of the hand rank is known, a key needs to be produced corresponding to the hand. The key is produced by converting the hand to an ```ìnt```. The conversion methodis determined as follows: 

* If the hand rank is in ```HandTracker.FLUSH_RANKS``` or ```HandTracker.UNIQUE5_RANKS``` then use ```HandTracker.TwosEncoding(hand)```.
* If the hand rank is in ```HandTracker.DUPES_RANKS``` then use ```HandTracker.PrimesEncoding(hand)```.

The ```HandTracker.TwosEncoding()``` method involves bitwise-xor of the ```int(Card)``` values of the cards in the hand. The ```int(Card)``` value of a card is a binary encoding, shaped by a bitmask that assigns each unique card value to a power of two. As long as each card in the hand has a unique value, the result of bitwise-xor will produce an ```int``` with five on-bits that's unique for the hand regardless of card order in the hand. The encoding is not suitable for unique encodings of hands containing duplicate values.

In [25]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()

for card in hand:
    print(f"{card} value: {bin(card._value >> 16)[2:].zfill(13)}")
print(f"bitwise-xor: {bin(tracker.TwosEncoding(hand))[2:].zfill(13)}")
print(f"Twos Encoding: {tracker.TwosEncoding(hand)}")

9♣ value: 0000010000000
8♠ value: 0000001000000
A♢ value: 1000000000000
7♢ value: 0000000100000
4♣ value: 0000000000100
bitwise-xor: 1000011100100
Twos Encoding: 4324


The ```HandTracker.PrimesEncoding()``` method involves multiplication of the ```int(Card)``` values of the cards in the hand. The ```int(Card)``` value of a card is a binary encoding, shaped by a bitmask that assigns each unique card value to a prime number. The result of multiplication will always produce a unique ```int``` for the hand regardless of card order in the hand, or the suits of the cards. 

In [26]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()

for card in hand:
    print(f"{card} prime: {card._prime}")
print(f"Primes Encoding: {tracker.PrimesEncoding(hand)}")

10♠ prime: 23
Q♡ prime: 31
A♣ prime: 41
5♣ prime: 7
5♠ prime: 7
Primes Encoding: 1432417


For more information about how ```int(Card)``` is calculated see the [card.ipynb](card.ipynb) notebook.

### **Evaluating Hands of Tracked Players**

The ```HandTracker.EvaluatePlayersIn()``` method calls ```HandTracker.EvaluateHand()``` for each player in ```HandTracker.TrackedPlayers()``` and stores the results in the ```HandTracker.players``` attribute.

In [32]:
tracker = HandTracker()
names = ["Brad", "Scott", "Dan", "Alex", "Olivia", "Lewis"]
tracker.TrackPlayers(names)

tracker.ShuffleDeck()
tracker.DealPlayersIn()
tracker.EvaluatePlayersIn()
print(f"player hands: {tracker.players}")

player hands: {'Brad': {'cards': [A♢, 10♢, 2♠, A♠, 3♣], 'rank_n': 3489, 'rank_c': 'pair'}, 'Scott': {'cards': [3♡, K♣, 5♡, J♠, 4♠], 'rank_n': 6878, 'rank_c': 'high card'}, 'Dan': {'cards': [4♣, 6♢, 7♡, 7♠, A♣], 'rank_n': 4912, 'rank_c': 'pair'}, 'Alex': {'cards': [8♠, J♢, Q♣, 8♢, 8♣], 'rank_n': 2027, 'rank_c': 'three of a kind'}, 'Olivia': {'cards': [10♣, 7♢, 5♢, 10♠, 9♢], 'rank_n': 4377, 'rank_c': 'pair'}, 'Lewis': {'cards': [5♣, 2♡, J♣, K♡, K♠], 'rank_n': 3678, 'rank_c': 'pair'}}


## **Swapping Cards**

The ```HandTracker.AllowDiscards(hand, discards)``` method is used to check if the rules of five card draw allow the selection of discards for the given hand. The rules are roughly:

* you can't discard all your cards
* if you discard four, the remaining card must be an ace.

In [3]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()
discards = hand[:4]

print(f"hand: {hand}")
print(f"discard request: {discards}")

if tracker.AllowDiscards(hand, discards):
    print(f"response: Allowed")
else:
    print(f"response: Disallowed")

hand: [J♡, J♢, K♠, J♠, 4♣]
discard request: [J♡, J♢, K♠, J♠]
response: Disallowed


The ```HandTracker.SwapCards(cards)``` methods provides new cards if there is enough in the deck.

In [5]:
tracker = HandTracker()
tracker.ShuffleDeck()

hand = tracker.DealHand()
print(f"old cards: {hand}")
new_cards = tracker.SwapCards(hand)
print(f"new cards: {new_cards}")

old cards: [8♡, 2♡, 3♢, 7♣, 7♢]
new cards: [Q♣, 10♡, 8♠, J♣, 6♢]


For tracked players use ```HandTracker.SwapPlayersCards(name, discards)```, which stores the updated hand in ```HandTracker.players```.

In [3]:
tracker = HandTracker()
names = ["Brad"]
tracker.TrackPlayers(names)

tracker.ShuffleDeck()
tracker.DealPlayersIn()
print(f"before swapping: {tracker.players}")

tracker.SwapPlayersCards(names[0], tracker.TrackedHand(names[0]))
print(f"after swapping: {tracker.players}")

before swapping: {'Brad': {'cards': [Q♣, 5♢, 9♡, 3♠, J♠]}}
after swapping: {'Brad': {'cards': [A♡, 8♠, K♣, 2♣, A♠]}}


## **Collecting Cards**

The recommended way to collect cards from players is to use the ```HandTracker.CollectCards()``` method which clears ```HandTracker.players``` and prepares ```HandTracker.DECK``` for a new round.